In [11]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
from numpy import random
import matplotlib.pyplot as plt

In [12]:
# Create quantum and classical registers
alice_qubit = QuantumRegister(1, "alice")       # Alice's qubit in the entangled pair
bob_qubit = QuantumRegister(1, "bob")           # Bob's qubit in the entangled pair
random_bit = QuantumRegister(1, "random")       # Random bit generator qubit
alice_classical = ClassicalRegister(2, "alice_bits")  # Stores c and d for verification
bob_classical = ClassicalRegister(2, "bob_bits")      # Measurement results for comparison

In [13]:
# Quantum circuit for superdense coding
circuit = QuantumCircuit(alice_qubit, bob_qubit, random_bit, alice_classical, bob_classical)

# Step 1: Entangle Alice's and Bob's qubits

In [ ]:

circuit.h(alice_qubit)
circuit.cx(alice_qubit, bob_qubit)
circuit.barrier()

CircuitInstruction(operation=Instruction(name='barrier', num_qubits=3, num_clbits=0, params=[]), qubits=(Qubit(QuantumRegister(1, 'alice'), 0), Qubit(QuantumRegister(1, 'bob'), 0), Qubit(QuantumRegister(1, 'random'), 0)), clbits=())

# Step 2: Generate random classical bits c and d using the random qubit

In [ ]:

# Measure the random qubit in superposition to randomly generate c and d
circuit.h(random_bit)
circuit.measure(random_bit, alice_classical[0])
circuit.h(random_bit)  # Apply another H to random_bit for second random value
circuit.measure(random_bit, alice_classical[1])
circuit.barrier()

CircuitInstruction(operation=Instruction(name='barrier', num_qubits=3, num_clbits=0, params=[]), qubits=(Qubit(QuantumRegister(1, 'alice'), 0), Qubit(QuantumRegister(1, 'bob'), 0), Qubit(QuantumRegister(1, 'random'), 0)), clbits=())

# Step 3: Encode Alice's message using the values of c and d

In [ ]:

# Conditionally apply gates based on random values of c and d
with circuit.if_test((alice_classical[0], 1)):
    circuit.x(alice_qubit)
with circuit.if_test((alice_classical[1], 1)):
    circuit.z(alice_qubit)
circuit.barrier()

CircuitInstruction(operation=Instruction(name='barrier', num_qubits=3, num_clbits=0, params=[]), qubits=(Qubit(QuantumRegister(1, 'alice'), 0), Qubit(QuantumRegister(1, 'bob'), 0), Qubit(QuantumRegister(1, 'random'), 0)), clbits=())

# Step 4: Bob decodes Alice's message

In [ ]:

circuit.cx(alice_qubit, bob_qubit)
circuit.h(alice_qubit)
circuit.barrier()


CircuitInstruction(operation=Instruction(name='barrier', num_qubits=3, num_clbits=0, params=[]), qubits=(Qubit(QuantumRegister(1, 'alice'), 0), Qubit(QuantumRegister(1, 'bob'), 0), Qubit(QuantumRegister(1, 'random'), 0)), clbits=())

# Step 5: Measure to retrieve c and d from Bob's qubits

In [ ]:

circuit.measure(alice_qubit, bob_classical[0])
circuit.measure(bob_qubit, bob_classical[1])

In [19]:
# Run the simulation
simulator = AerSimulator()
result = simulator.run(circuit).result()
counts = result.get_counts()

In [24]:
# Display the circuit and results
print("Quantum Circuit:")
print(circuit)
print("\nMeasurement Counts:")
print(counts)
plot_histogram(counts)
plt.show()


Quantum Circuit:
              ┌───┐      ░                  ░       ┌──────       ┌───┐»
       alice: ┤ H ├──■───░──────────────────░───────┤ If-0  ──────┤ X ├»
              └───┘┌─┴─┐ ░                  ░       └──╥───       └───┘»
         bob: ─────┤ X ├─░──────────────────░──────────╫───────────────»
                   └───┘ ░ ┌───┐┌─┐┌───┐┌─┐ ░          ║               »
      random: ───────────░─┤ H ├┤M├┤ H ├┤M├─░──────────╫───────────────»
                         ░ └───┘└╥┘└───┘└╥┘ ░ ┌────────╨─────────┐     »
alice_bits: 2/═══════════════════╩═══════╩════╡ alice_bits_0=0x1 ╞═════»
                                 0       1    └──────────────────┘     »
  bob_bits: 2/═════════════════════════════════════════════════════════»
                                                                       »
«               ───────┐       ┌──────       ┌───┐ ───────┐  ░      ┌───┐ ░ ┌─┐»
«       alice:   End-0 ├───────┤ If-0  ──────┤ Z ├  End-0 ├──░───■──┤ H ├─░─┤M├»
«               ──

The output counts confirms that Alice and Bob's classical bits match, which indicates superdense coding protocol is working correctly